

Para aquel contenido georeferenciado publicado anónimamente indicar por país, cuántas IPs de usuarios corresponden a IPv4 y cuantas a IPv6. (⭐)

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=8471ed01db992d1b35476745d3dd778652d25a1cc561a04f57ee2dc64422e9f2
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/c

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext_1 = SQLContext(sc)
df_geografico = sqlContext_1.read.csv('/content/drive/MyDrive/geo_tags.csv', header=True, inferSchema=True, multiLine=True, escape='"')
rdd_geografico = df_geografico.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
sqlContext_2 = SQLContext(sc)
df_contenido = sqlContext_2.read.csv('/content/drive/MyDrive/contents.csv', header=True, inferSchema=True, multiLine=True, escape='"')
rdd_contenido = df_contenido.rdd

Elimino las columnas que no uso

In [ ]:
rdd_geo_uso = rdd_geografico.map(lambda x: (x.gt_page_id, x.gt_country))

In [ ]:
rdd_cont_uso = rdd_contenido.map(lambda x: (x.id, x.revisor_ip))

Hago un Join entre los rdd_geo_uso y rdd_cont_uso

In [ ]:
rdd_join = rdd_cont_uso.join(rdd_geo_uso)

Utilizo en este formato los datos

In [ ]:
rdd_join = rdd_join.map(lambda x: (x[0], x[1][0], x[1][1]))

In [ ]:
rdd_join = rdd_join.filter(lambda x: (x[1] != None))

In [ ]:
rdd_join = rdd_join.filter(lambda x: (x[2] != None))

Defino estas funciones para identificar la version de la ip, dependiendo de si hay : o no.

In [ ]:
def identificar_ipv4(x):

  encontro = 0

  for y in range(0, len(x)):
    if(x[y] == ':'):
        encontro = 1
  return encontro

In [ ]:
def identificar_ipv6(x):

  encontro = 0

  for y in range(0, len(x)):
    if(x[y] != ':'):
        encontro = 1
    else:
        return 0
  return encontro

In [ ]:
rdd_join = rdd_join.map(lambda x: (x[2], x[1]))

Cambio el formato, si es v4 va primero y v6 segundo

In [ ]:
rdd_join = rdd_join.map(lambda x: (x[0], (identificar_ipv4(x[1]), identificar_ipv6(x[1]))))

In [ ]:
rdd_join.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])).collect()